# L4: Support Data Insight Analysis

<p style="background-color:#fff6e4; padding:15px; border-width:3px; border-color:#f5ecda; border-style:solid; border-radius:6px"> ⏳ <b>Note <code>(Kernel Starting)</code>:</b> This notebook takes about 30 seconds to be ready to use. You may start and watch the video while you wait.</p>

## Initial Imports

In [1]:
__import__('pysqlite3')
import sys
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')


In [2]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

# Load environment variables
from helper import load_env
load_env()

import os
import yaml
from crewai import Agent, Task, Crew,LLM

<p style="background-color:#fff6ff; padding:15px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px"> 💻 &nbsp; <b>Access <code>requirements.txt</code> and <code>helper.py</code> files:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>. For more help, please see the <em>"Appendix - Tips and Help"</em> Lesson.</p>

## Loading Tasks and Agents YAML files

In [3]:
INFERENCE_SERVER_URL = "http://localhost:8989"
MODEL_NAME = "ibm-granite/granite-3.3-2b-instruct"
API_KEY= "alanliuxiang"


llm = LLM(model="ibm-granite/granite-3.3-2b-instruct", 
          base_url=f"{INFERENCE_SERVER_URL}/v1",
          api_key=API_KEY,
          custom_llm_provider ="openai",
          max_tokens=1024)

In [4]:
# Define file paths for YAML configurations
files = {
    'agents': 'L4-config/agents.yaml',
    'tasks': 'L4-config/tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs['agents']
tasks_config = configs['tasks']

## Using FileReadTool

In [5]:
from crewai_tools import FileReadTool
csv_tool = FileReadTool(file_path='./support_tickets_data.csv')

/opt/app-root/lib64/python3.11/site-packages/pydantic/fields.py:1093: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'required'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warn(


## Creating Agents, Tasks and Crew

In [6]:
from crewai_tools import CodeInterpreterTool
# Creating Agents
suggestion_generation_agent = Agent(
  config=agents_config['suggestion_generation_agent'],
  llm=llm,
  tools=[csv_tool]
)

reporting_agent = Agent(
  config=agents_config['reporting_agent'],
  llm=llm,
  tools=[csv_tool]
)

chart_generation_agent = Agent(
  config=agents_config['chart_generation_agent'],
  tools=[CodeInterpreterTool(unsafe_mode=True)],
  llm=llm,
  # allow_code_execution=True,
  code_execution_mode="unsafe",
)

# Creating Tasks
suggestion_generation = Task(
  config=tasks_config['suggestion_generation'],
  agent=suggestion_generation_agent
)

table_generation = Task(
  config=tasks_config['table_generation'],
  agent=reporting_agent
)

chart_generation = Task(
  config=tasks_config['chart_generation'],
  agent=chart_generation_agent
)

final_report_assembly = Task(
  config=tasks_config['final_report_assembly'],
  agent=reporting_agent,
  context=[suggestion_generation, table_generation, chart_generation]
)


# Creating Crew
support_report_crew = Crew(
  agents=[
    suggestion_generation_agent,
    reporting_agent,
    chart_generation_agent
  ],
  tasks=[
    suggestion_generation,
    table_generation,
    chart_generation,
    final_report_assembly
  ],
  verbose=True
)


## Testing our Crew

In [7]:
# support_report_crew.test(n_iterations=1, openai_model_name='gpt-4o')
# support_report_crew.test(n_iterations=1)

support_report_crew.test(n_iterations=1,eval_llm=llm)

2025-07-16 04:11:30,337 - 140296615413568 - __init__.py-__init__:111 - WARNING: Invalid type LLM for attribute 'model_name' value. Expected one of ['bool', 'str', 'bytes', 'int', 'float'] or a sequence of those types


╭──────────────────────────────────────────────── Test Execution ─────────────────────────────────────────────────╮
│                                                                                                                 │
│  🧪 Starting Crew Test                                                                                          │
│                                                                                                                 │
│  Crew: crew                                                                                                     │
│  ID: 0d28c3dc-67d0-463c-9d8f-0b4d1c545a7a                                                                       │
│  Iterations: 1                                                                                                  │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Provider List: https://docs.litellm.ai/docs/providers

Provider List: https://docs.litellm.ai/docs/providers

Provider List: https://docs.litellm.ai/docs/providers

...

[EventBus Error] Handler 'on_crew_test_result' failed for event 'CrewTestResultEvent': 'Crew' object has no attribute 'crew'


╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: e14d1764-9850-4bbf-81a0-e3b0f86a4c70                                                                     │
│  Agent: Report Generator                                                                                        │
│                                                                                                                 │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 7609cbd2-f1a4-4172-b0f7-22b8d0340c11                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: ### Support System Performance Summary Report                                                    │
│                                                                                                                 │
│  #### Issue Classification Results                                                                              │
│                                                                                                                 │
│  | Issue Classification Results                                          |                                      │
│  |----------------pmatrix| Frequency and Priority                          |                                    │
│  |----------------------------------------------------------------|-------------------------------------------  │
│  -----_|                                                                                                        │
│  | Malfunctioning Features | 21, High (5)                          |                                            │
│  | Data Import/Export Issues | 20, Medium (4)                         |                                         │
│  | Access/Login Problems |                                   |                                                  │
│  | Report Generation Challenges | 18, Low (3)                         |                                         │
│  | General Request/Feature Request | 22, Medium (5)                    |                                        │
│  | Billing/Payment Discrepancies | 19, High (4)                        |                                        │
│  | UI/User Experience Issues | 17, Medium (4)                        |                                          │
│  | Escalated Issues |                                   |                                                       │
│  | Duplicate Issue Reporting |  8, Low (2)                          |                                           │
│                                                                                                                 │
│  ![issue_distribution](issue_distribution.png)                                                                  │
│  - Issue Type distribution chart highlighting the frequency and high/medium/low priority categorization.        │
│                                                                                                                 │
│  #### Agent Performance                                                                                         │
│                                                                                                                 │
│  | Agent_ID | Average Response Time (mins) | Average Resolution Time (mins) | Average Satisfaction Rating |     │
│  |----------|-----------------------------|-------------------------------|--------------------------------|    │
│  | A004     | 200.5                          | 250.3                         | 4.1                              │
│  |                                                                                                              │
│  | A002     | 185.2                          | 245.6                         | 3.9                              │
│  |                                                    

                             Tasks Scores                              
                        (1-10 Higher is better)                        
┏━━━━━━━━━━━━━━━━━━━━┯━━━━━━━┯━━━━━━━━━━━━┯━━━━━━━━━━━━━━━━━━━━━┯━━┯━━┓
┃ Tasks/Crew/Agents  │ Run 1 │ Avg. Total │ Agents              │  │  ┃
┠────────────────────┼───────┼────────────┼─────────────────────┼──┼──┨
┃ Task 1             │  9.0  │    9.0     │ - Suggestion Engine │  │  ┃
┃                    │       │            │                     │  │  ┃
┃                    │       │            │                     │  │  ┃
┃ Task 2             │  9.0  │    9.0     │ - Report Generator  │  │  ┃
┃                    │       │            │                     │  │  ┃
┃                    │       │            │                     │  │  ┃
┃ Task 3             │ 10.0  │    10.0    │ - Chart Specialist  │  │  ┃
┃                    │       │            │                     │  │  ┃
┃                    │       │            │                     │  │  ┃
┃ Task 4             │  9.0  │    9.0     │ - Report Generator  │  │  ┃
┃                    │       │            │                     │  │  ┃
┃ Crew               │ 9.25  │    9.2     │                     │  │  ┃
┃ Execution Time (s) │  98   │     98     │                     │  │  ┃
┗━━━━━━━━━━━━━━━━━━━━┷━━━━━━━┷━━━━━━━━━━━━┷━━━━━━━━━━━━━━━━━━━━━┷━━┷━━┛

╭──────────────────────────────────────────────── Test Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Test Execution Completed                                                                                       │
│  Crew: crew                                                                                                     │
│                                                                                                                 │
│  Status: Completed                                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

## Training your crew and agents

In [8]:
support_report_crew.train(n_iterations=1, filename='training.pkl')

╭─────────────────────────────────────────────── Training Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  📋 Crew Training Started                                                                                       │
│  Crew: crew                                                                                                     │
│  Time: 2025-07-16 04:13:23.954704                                                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 83b4eb05-7352-413e-861d-15566ffce604                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Provider List: https://docs.litellm.ai/docs/providers

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Suggestion Engine                                                                                       │
│                                                                                                                 │
│  Task: Generate actionable suggestions for resolving each classified support ticket. The suggestions should be  │
│  based on: - Issue Type: Tailor suggestions to the specific type of issue reported. - Historical Data: Use      │
│  historical data such as resolution_time_minutes and                                                            │
│    satisfaction_rating to inform the suggestions.                                                               │
│  - Customer Feedback: Incorporate insights from customer_comments to                                            │
│    customize the suggestions further.                                                                           │
│                                                                                                                 │
│  The goal is to provide clear, actionable steps that the support team can take to resolve each issue            │
│  efficiently and effectively.                                                                                   │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

...

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Error: File not found at path: ./support_data.csv                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Report Generator                                                                                        │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  As there's an error indicating the file './support_data.csv' couldn't be found, it’s crucial to ensure the     │
│  correct file path is provided. Assuming the file exists, here's how the analysis and visualization would       │
│  proceed:                                                                                                       │
│                                                                                                                 │
│  ```python                                                                                                      │
│  import pandas as pd                                                                                            │
│  import matplotlib.pyplot as plt                                                                                │
│  import seaborn as sns                                                                                          │
│  from datetime import datetime                                                                                  │
│                                                                                                                 │
│  # Reading the support_data.csv file                                                                            │
│  support_data = pd.read_csv('./support_data.csv')                                                               │
│                                                                                                                 │
│  # Issue Classification Results                                                                                 │
│  issue_type_distribution = support_data['issue_type'].value_counts().sort_index()                               │
│                                                                                                                 │
│  plt.figure(figsize=(10,6))                                                                                     │
│  issue_type_distribution.plot(kind='bar', title='Issue Distribution', color='skyblue')                          │
│  plt.xlabel('Issue Type')                                                                                       │
│  plt.ylabel('Count')                                                                                            │
│  plt.savefig('issue_distribution.png')                                                                          │
│                                                                                                                 │
│  # Priority Levels                                                                                              │
│  priority_breakdown = support_data['priority'].value_counts(normalize=True)                                     │
│  priority_breakdown.plot(kind='bar', rot=0, title='Priority Levels Breakdown', color='lightgreen')              │
│  plt.xlabel('Priority Level')                                                                                   │
│  plt.ylabel('Proportion')                                                                                       │
│  plt.savefig('priority_levels.png')                                                                             │
│                                                                                                                 │
│  # Agent Performance                                   

 ## Final Result: As there's an error indicating the file './support_data.csv' couldn't be found, it’s crucial to ensure the correct file path is provided. Assuming the file exists, here's how the analysis and visualization would proceed:

```python
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# Reading the support_data.csv file
support_data = pd.read_csv('./support_data.csv')

# Issue Classification Results
issue_type_distribution = support_data['issue_type'].value_counts().sort_index()

plt.figure(figsize=(10,6))
issue_type_distribution.plot(kind='bar', title='Issue Distribution', color='skyblue')
plt.xlabel('Issue Type')
plt.ylabel('Count')
plt.savefig('issue_distribution.png')

# Priority Levels
priority_breakdown = support_data['priority'].value_counts(normalize=True)
priority_breakdown.plot(kind='bar', rot=0, title='Priority Levels Breakdown', color='lightgreen')
plt.xlabel('Priority Level')
plt.ylabel('Proportion')
plt.

KeyboardInterrupt: Interrupted by user

## Comparing new test results

In [ ]:
# support_report_crew.test(n_iterations=1, openai_model_name='gpt-4o')
support_report_crew.test(n_iterations=1,eval_llm=llm)
# support_report_crew.test(n_iterations=1)

In [ ]:
# Display the Trello screenshot
from IPython.display import Image, display

# Load and display the image
test_image = Image(filename='test_before_training.png', width=368)
display(test_image)

## Kicking off Crew

In [ ]:
result = support_report_crew.kickoff()

## Result

In [ ]:
from IPython.display import display, Markdown
display(Markdown(result.raw))